# LangChain Agent 使用 function call 用非结构化数据自动创建订单

* 先看一下 OpenAI API 的 [function-call-example](./openai_example.ipynb) 示例
* 基于 OpenAI API 实现自动创建订单 [openai-auto-create-order](./openai_auto_create_order.ipynb)
* 基于 langchain 实现自动创建订单 [langchain-auto-create-order](./langchain_auto_create_order.ipynb)

## 背景
客户为了快速下单，会把订单信息先录入 Excel 文档，然后通过 Excel 文档导入的方式批量创建订单。
但是不同客户可能有自己的订单格式，比如收件人姓名、收件人电话等，这些信息在客户一方的列名、顺序可能不一样。
但为了下单，需要把信息转换为我们的订单格式。
以前几乎都是通过人工的方式来适配我们的订单格式。

## 解决方案
这是一个典型的数据转换问题，可以通过命名实体识别（NER）来解决。
流程是：
1. 通过命名实体识别抽取数据中特定的参数
2. 用命名实体识别抽取的参数创建订单

在以前，要进行 NER，需要标注数据，然后训练专有模型。

现在可以通过 LLM(OpenAI API) 来解决，只需要提供示例与数据，OpenAI API 就可以自动完成 NER。

并且，在 2023-06-13 发布的 [gpt-3.5-turbo-16k-0613](https://beta.openai.com/models/gpt3/versions/3.5-turbo-16k-0613) 模型中，OpenAI API 支持 function call，并且能自动提取非结构化数据，然后大模型可以自行分析应该调用哪个 function。

这样就能做到载入客户数据（Excel、CSV、JSON、XML、自然语言等），然后通过 function call 自动调用软件已有的功能，完成订单创建。

## 进一步思考
从现在的测试来看，微调过后的新版本，能自动做 NER 然后自动选择要调用的 function。

这里核心解决了两个问题：
1. 信息识别与转换
2. 信息链接

这里有很大的想象空间

In [1]:
# 所有缺少的包都可以通过 !pip -q install xxx 安装。-q 可以减少日志输出
# !pip -q install openai --upgrade
# !pip -q install langchain --upgrade

在 .env 中配置相应参数

# 使用 agent

### 准备非结构化数据

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
import langchain

langchain.debug=True

file_path = "./test-order.csv"
loader = CSVLoader(file_path)
data = loader.load()
print(data)
# 每一个 page_content 就是一个字符串
texts = [doc.page_content for doc in data]

[Document(page_content='客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: ', metadata={'source': './test-order.csv', 'row': 0}), Document(page_content='客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: ', metadata={'source': './test-order.csv', 'row': 1})]


In [3]:
# 额外增加一些非结构化数据
texts = '。\n'.join(texts) + '。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。'
print(texts)

客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 运荔枝
收件人电话号码: 12345678900
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。
客户订单号: GO202001014
线路标识: 
收货地址: 火炬动力港
收件人姓名: 鲜生活
收件人电话号码: 13409871234
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。


### 准备 agent 需要的 tool

In [4]:
from langchain import PromptTemplate
from langchain.tools import tool

# 在 function description 中描述参数结构，LLM 可能会自动识别，但最好还是配置 args_schema
@tool("create_order")
def create_order(orders):
    """用给定的参数创建订单 参数为 [{'receiverName': '', 'receiverMobile': ''}]"""
    for order_info in orders:
        receiver_name = order_info["receiverName"]
        receiver_mobile = order_info["receiverMobile"]
        print(f'create order for  {receiver_name}/{receiver_mobile}\n')
    return f"{orders}\n订单创建成功!"

print(create_order.args_schema.schema())

# 工具是给 agent 自动分析使用
tools = [create_order]

{'title': 'create_orderSchemaSchema', 'type': 'object', 'properties': {'orders': {'title': 'Orders'}}, 'required': ['orders']}


如果给两个 tool，agent 衔接还有些问题。
```
先做数据抽取 do_ner 返回的数据非常标准。
[{'receiverName': '运荔枝', 'receiverMobile': '12345678900'}, {'receiverName': '鲜生活', 'receiverMobile': '13409871234'}, {'receiverName': '刘德华', 'receiverMobile': '13400000000'}, {'receiverName': '张学友', 'receiverMobile': '13500000000'}, {'receiverName': '黎明', 'receiverMobile': '13600000000'}, {'receiverName': '郭富城', 'receiverMobile': '13700000000'}]

但是 agent 在调用 create_order 时并没有把第一个函数的结果给第二个函数。这个还需要测试。
Invoking: `create_order` with `{'orders': [{'receiverName': '', 'receiverMobile': ''}]}`
```

### 准备 问题 与 agent，执行任务

In [5]:
question = """
请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{{"receiverName": "", "receiverMobile": ""}}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
{texts}
'''
"""
question_prompt = PromptTemplate.from_template(question)
question_format_str = question_prompt.format(texts=texts)
print(question_format_str)


请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{"receiverName": "", "receiverMobile": ""}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 运荔枝
收件人电话号码: 12345678900
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。
客户订单号: GO202001014
线路标识: 
收货地址: 火炬动力港
收件人姓名: 鲜生活
收件人电话号码: 13409871234
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。
'''



In [6]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from learning_langchain.multi_key_chat_open_ai import MultiKeyChatOpenAI

llm = MultiKeyChatOpenAI(temperature=0, verbose=True)

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True)
agent_response = agent.run(question_format_str)
print(agent_response)

[chain/start] [1:RunTypeEnum.chain:AgentExecutor] Entering Chain run with input:
{
  "input": "\n请根据给出的信息抽取相应的参数创建订单\n1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：\n[{\"receiverName\": \"\", \"receiverMobile\": \"\"}]\n2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单\n\n输入信息：\n'''\n客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。\n客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。\n'''\n"
}
[llm/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.llm:MultiKeyChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a hel

# 不使用 agent

langchain 是通过 tool 来包装 function calling 的

### 定义函数，准备 tool

In [7]:
from pydantic import BaseModel
from langchain.tools import Tool, format_tool_to_openai_function
from langchain.schema import SystemMessage, HumanMessage

def do_ner(ner_input: str):
    """通过命名实体识别抽取数据中特定格式实体信息"""
    ner_output_template = """ [
        {
            "receiverName": "张三",
            "receiverMobile": "13800000000"
        },
        {
            "receiverName": "李四",
            "receiverMobile": "13800000001"
        }
     ]"""

    template = """
    请严格按照抽取格式返回数据，不得抽取多的实体信息，如果指定的实体信息缺失，可以设置空值。
    抽取格式：
    {ner_template}
    数据：
    {texts}
    """

    prompt = PromptTemplate.from_template(template)
    format_str = prompt.format(ner_template=ner_output_template, texts=ner_input)

    messages = [
        SystemMessage(
            content="你是一个命名实体识别模型。我会给你抽取格式与数据，请根据示例格式抽取数据中的信息返回。"
        ),
        HumanMessage(
            content=format_str
        ),
    ]
    results = llm.predict_messages(messages)
    content = results.content
    return content

def create_order(orders):
    """用给定的参数创建订单 参数为 [{'receiverName': '', 'receiverMobile': ''}]"""
    for order_info in orders:
        receiver_name = order_info["receiverName"]
        receiver_mobile = order_info["receiverMobile"]
        print(f'create order for  {receiver_name}/{receiver_mobile}\n')
    return f"{orders}\n订单创建成功!"

class DoNerInput(BaseModel):
    ner_input: str

class Order(BaseModel):
    receiverName: str
    receiverMobile: str

class CreateOrderInput(BaseModel):
    orders: list[Order]

# 测试时发现，在 description 描述参数结构，一定程度可以让 OpenAI 识别参数结构，但不稳定，因此最好是在 args_schema 中定义参数结构。
new_tools = [
    Tool.from_function(
        name='do_ner',
        description='通过命名实体识别抽取数据中特定格式实体信息',
        func=do_ner,
        args_schema=DoNerInput
    ),
    Tool.from_function(
        name='create_order',
        description="用给定的参数创建订单 参数为 orders: [{'receiverName': '', 'receiverMobile': ''}]",
        func=create_order,
        args_schema=CreateOrderInput
    ),
]


functions = [format_tool_to_openai_function(t) for t in new_tools]
print(functions)

[{'name': 'do_ner', 'description': '通过命名实体识别抽取数据中特定格式实体信息', 'parameters': {'title': 'DoNerInput', 'type': 'object', 'properties': {'ner_input': {'title': 'Ner Input', 'type': 'string'}}, 'required': ['ner_input']}}, {'name': 'create_order', 'description': "用给定的参数创建订单 参数为 orders: [{'receiverName': '', 'receiverMobile': ''}]", 'parameters': {'title': 'CreateOrderInput', 'type': 'object', 'properties': {'orders': {'title': 'Orders', 'type': 'array', 'items': {'$ref': '#/definitions/Order'}}}, 'required': ['orders'], 'definitions': {'Order': {'title': 'Order', 'type': 'object', 'properties': {'receiverName': {'title': 'Receivername', 'type': 'string'}, 'receiverMobile': {'title': 'Receivermobile', 'type': 'string'}}, 'required': ['receiverName', 'receiverMobile']}}}}]


### 定义执行本地函数的函数

In [8]:
# 获取本地函数的参数，执行本地函数
import inspect
import json

def get_function_parameter_names(function):
  isfunction = inspect.isfunction(function)
  if function is not None and isfunction:
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

# Locate the function and make the call
def cal_function(_function_name, _arguments):
    the_function = globals().get(_function_name)
    parameter_names = get_function_parameter_names(the_function)
    parameter_values = []
    for parameter_name in parameter_names:
      parameter_values.append(_arguments[parameter_name])
    return the_function(*parameter_values)

### 执行 function calling，并根据 LLM function calling 的返回值，调用本地函数

In [9]:
message = llm.predict_messages(
    [HumanMessage(content=question_format_str)], functions=functions
)
print(message)

[llm/start] [1:RunTypeEnum.llm:MultiKeyChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n请根据给出的信息抽取相应的参数创建订单\n1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：\n[{\"receiverName\": \"\", \"receiverMobile\": \"\"}]\n2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单\n\n输入信息：\n'''\n客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。\n客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。\n'''"
  ]
}
[llm/end] [1:RunTypeEnum.llm:MultiKeyChatOpenAI] [20.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
 

In [10]:
# 根据 LLM function calling 的返回值，调用函数
function_kw = message.additional_kwargs['function_call']
function_name = function_kw['name']
function_arguments = json.loads(function_kw['arguments'])
returned_value = cal_function(function_name, function_arguments)
print(returned_value)

[llm/start] [1:RunTypeEnum.llm:MultiKeyChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 你是一个命名实体识别模型。我会给你抽取格式与数据，请根据示例格式抽取数据中的信息返回。\nHuman: \n    请严格按照抽取格式返回数据，不得抽取多的实体信息，如果指定的实体信息缺失，可以设置空值。\n    抽取格式：\n     [\n        {\n            \"receiverName\": \"张三\",\n            \"receiverMobile\": \"13800000000\"\n        },\n        {\n            \"receiverName\": \"李四\",\n            \"receiverMobile\": \"13800000001\"\n        }\n     ]\n    数据：\n    客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。\n客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两

### 根据本地函数的执行结果，再次执行 function calling，并根据 LLM function calling 的返回值，调用本地函数

In [11]:
from langchain.schema import AIMessage, ChatMessage

# 根据第一次调用函数的返回值，让 LLM 继续回答
second_message = llm.predict_messages(
    [
        HumanMessage(content=question_format_str),
        # 把 LLM function calling 返回的 function 信息作为 AIMessage 的 content
        AIMessage(content=str(message.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ],
    functions=functions
)
print(second_message)

[llm/start] [1:RunTypeEnum.llm:MultiKeyChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n请根据给出的信息抽取相应的参数创建订单\n1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：\n[{\"receiverName\": \"\", \"receiverMobile\": \"\"}]\n2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单\n\n输入信息：\n'''\n客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。\n客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。\n'''\n\nAI: {'function_call': {'name': 'do_ner', 'arguments': '{\\n\"ner_input\": \"客户订单号: GO202001013\\\\n线路标识: \\\\n收货地址: 火炬动力港\\\\n收件人姓名: 运荔枝\\\\n收件人电话号

In [12]:
# 再次根据 LLM function calling 的返回值，继续调用函数
function_kw = second_message.additional_kwargs['function_call']
function_name = function_kw['name']
function_arguments = json.loads(function_kw['arguments'])
returned_value = cal_function(function_name, function_arguments)
print(returned_value)

create order for  运荔枝/12345678900

create order for  鲜生活/13409871234

create order for  刘德华/13400000000

create order for  张学友/13500000000

create order for  黎明/13600000000

create order for  郭富城/13700000000

[{'receiverName': '运荔枝', 'receiverMobile': '12345678900'}, {'receiverName': '鲜生活', 'receiverMobile': '13409871234'}, {'receiverName': '刘德华', 'receiverMobile': '13400000000'}, {'receiverName': '张学友', 'receiverMobile': '13500000000'}, {'receiverName': '黎明', 'receiverMobile': '13600000000'}, {'receiverName': '郭富城', 'receiverMobile': '13700000000'}]
订单创建成功!


### 根据第二次执行本地函数的结果，让 LLM 给出最初问题的回答

In [13]:
# 根据执行结果，让 LLM 得出问题回答
result = llm.predict_messages(
    [
        HumanMessage(content=question_format_str),
        AIMessage(content=str(message.additional_kwargs)),
        AIMessage(content=str(second_message.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ], functions=functions
)
print(result)

[llm/start] [1:RunTypeEnum.llm:MultiKeyChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \n请根据给出的信息抽取相应的参数创建订单\n1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：\n[{\"receiverName\": \"\", \"receiverMobile\": \"\"}]\n2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单\n\n输入信息：\n'''\n客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。\n客户订单号: GO202001014\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。\n'''\n\nAI: {'function_call': {'name': 'do_ner', 'arguments': '{\\n\"ner_input\": \"客户订单号: GO202001013\\\\n线路标识: \\\\n收货地址: 火炬动力港\\\\n收件人姓名: 运荔枝\\\\n收件人电话号